## References
- https://medium.com/scrapehero/exploring-image-similarity-approaches-in-python-b8ca0a3ed5a3

In [1]:
import cv2 as cv
import numpy as np
from skimage import metrics

DELAY_CAPTION = 1500
DELAY_BLUR = 100
MAX_KERNEL_LENGTH = 31
SHOULD_DISPLAY = False

window_name = "Smoothing Demo"
cv.namedWindow(window_name, cv.WINDOW_AUTOSIZE)

def display_dst(delay, dst, name = window_name):
    if SHOULD_DISPLAY:
        cv.imshow(name, dst)
        c = cv.waitKey(delay)
        if c >= 0 : return -1
        return 0
    else:
        # save image
        cv.imwrite(name + ".png", dst)




def display_caption(src, caption):
    dst = np.zeros(src.shape, src.dtype)

    rows, cols, _ch = src.shape
    cv.putText(dst, caption,
                (int(cols / 4), int(rows / 2)),
                cv.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255))
    return display_dst(DELAY_CAPTION)

### Read images 

In [2]:
image = cv.imread("./xray.png")
clear = cv.imread("./clear.png")
dst = np.copy(image)

### Denoising Functions

In [3]:
def apply_mean_filter(src):
    dst = cv.blur(src, (MAX_KERNEL_LENGTH, MAX_KERNEL_LENGTH))
    if display_dst(DELAY_BLUR, dst, "Mean",) != 0:
        return dst

In [4]:
def apply_gaussian_blur(src):
  dst = cv.GaussianBlur(src, (MAX_KERNEL_LENGTH, MAX_KERNEL_LENGTH), 0)
  if display_dst(DELAY_BLUR, dst, "Gaussian") != 0:
    return dst

In [5]:
def apply_median_filter(src):
    dst = cv.medianBlur(src, MAX_KERNEL_LENGTH)
    if display_dst(DELAY_BLUR, dst, "Median") != 0:
        return dst

In [6]:
def apply_bilateral_filter(src):
    dst = cv.bilateralFilter(src, MAX_KERNEL_LENGTH, MAX_KERNEL_LENGTH * 2, MAX_KERNEL_LENGTH / 2)
    if display_dst(DELAY_BLUR, dst, "Bilateral") != 0:
        return dst

## Use SSIM to evaluate the similarity of the image

In [7]:
def calculate_similarity(src, dst):

  # Convert images to grayscale
  image1_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
  image2_gray = cv.cvtColor(dst, cv.COLOR_BGR2GRAY)
  # Calculate SSIM
  ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
  return ssim_score[0]

  

In [8]:
bilateral_dst = apply_bilateral_filter(image)
mean_dst = apply_mean_filter(image)
gaussian_dst = apply_gaussian_blur(image)
median_dst = apply_median_filter(image)

In [9]:
bilateral_similarity = calculate_similarity(clear, bilateral_dst)
mean_similarity = calculate_similarity(clear, mean_dst)
gaussian_similarity = calculate_similarity(clear, gaussian_dst)
median_similarity = calculate_similarity(clear, median_dst)
default_similarity = calculate_similarity(clear, image)

print("Bilateral Similarity: ", bilateral_similarity)
print("Mean Similarity: ", mean_similarity)
print("Gaussian Similarity: ", gaussian_similarity)
print("Median Similarity: ", median_similarity)
print("No Filter Similarity: ", default_similarity)

cv.destroyAllWindows()

Bilateral Similarity:  0.44838959289787944
Mean Similarity:  0.20774828799463957
Gaussian Similarity:  0.32135118083922637
Median Similarity:  0.3042781463872747
No Filter Similarity:  0.39772297421514735


### Get best similarity

In [10]:
scores = [
	(
		"No Filter",
		default_similarity
	),
	(
		"Bilateral",
		bilateral_similarity
	),
	(
		"Mean",
		mean_similarity
	),
	(
		"Gaussian",
		gaussian_similarity
	),
	(
		"Median",
		median_similarity
	)
]

scores.sort(key=lambda x: x[1], reverse=True)

print("Best method: ", scores[0][0])
print("Best method score: ", scores[0][1])

print("Worst method: ", scores[-1][0])
print("Worst method score: ", scores[-1][1])


Best method:  Bilateral
Best method score:  0.44838959289787944
Worst method:  Mean
Worst method score:  0.20774828799463957


: 

The best denoising method for this case is the **bilateral filter**. I think this is the best method because it is the one that best preserves the edges of the image in contrast to the other methods. 